In [3]:
import pandas as pd
import os
import crontab

In [ ]:
# Расписание по crontab
# Delta загрузка, записывать дату время загрузки в RAW
# Расчет DATAMART на основе RAW, беря во внимание delta

In [4]:
ext = ".csv"

customer_table_name = "customer"
payments_table_name = "payments"
customer_totals_name = "customer_totals"

SOURCE = "example"
RAW = "raw"
DATAMART = "datamart"

In [5]:
customer_df = pd.read_csv(os.path.join(SOURCE,customer_table_name+ext))
customer_df.head()

,customer_id,full_name,tariff
0,1,Minahil Robins,red
1,2,Liliana Hernandez,blue
2,3,Kaleb Myers,red
3,4,Yasser Rudd,blue
4,5,Lubna Shaffer,red


In [6]:
payments_df = pd.read_csv(os.path.join(SOURCE,payments_table_name+ext))
payments_df.head()

,payment_id,customer_id,payment_timestamp,amount
0,1,1,2022-03-01 20:22:00,100
1,2,2,2022-03-02 10:00:00,200
2,3,3,2022-03-03 20:22:00,500
3,4,4,2022-03-11 20:22:00,50
4,5,5,2022-02-21 20:22:00,300


In [7]:
if not os.path.exists(RAW):
    os.mkdir(RAW)

customer_df.to_csv(os.path.join(RAW,customer_table_name+ext))
payments_df.to_csv(os.path.join(RAW,payments_table_name+ext))

In [34]:
from pandasql import sqldf

customer_df = pd.read_csv(os.path.join(RAW,customer_table_name+ext))
payments_df = pd.read_csv(os.path.join(RAW,payments_table_name+ext))

pysqldf = lambda q: sqldf(q, globals())

customer_total_df = pysqldf("""
                    select c.customer_id, sum(p.amount) as amount 
                    from customer_df as c
                    join payments_df as p on c.customer_id=p.customer_id
                    group by c.customer_id
                    """)

if not os.path.exists(DATAMART):
    os.mkdir(DATAMART)

customer_total_df.to_csv(os.path.join(DATAMART,customer_totals_name+ext))
customer_total_df.set_index('customer_id')



,amount
customer_id,
1,183
2,429
3,525
4,229
5,557
6,360
7,535
8,785
9,440
